# Exercise 6
## Random numbers
*** 

### Questions
How can I generate random numbers, and what are they useful for?

### Objectives
<div class=obj>
<ol>
    <li>Draw random numbers.</li>
    <li>Use random numbers to generate probability distributions.</li>
    <li>Learn the basics of numerical error propagation</li>
</ol>
    
<ul>
Revise:
    <li>Plotting (histograms);</li>
    <li>Fitting regressions;</li>
    <li>Defining functions;</li>
    <li>If else statements;</li>
    <li>Data readin.</li>
</ul>
</div>

### Independent coding
Error propagation through magmatic crystallisation temperature estimates.

## 6.1 Generating randomness
***



Let's jump right in and generate a random number, using the `random` library (__[see here for details](https://docs.python.org/3/library/random.html#module-random)__).

In [ ]:
import random as rand

#randint(a,b) will generate 
rand.randint(0,100)

Keep re-running the above cell, does the number change?

You might wonder how a computer is randomly generating a number.  In fact, computers are generally totally dertministic in their random number generation.  In this case, we are creating a list of numbers between 0 and 100, python is generating a pseudo-random number between 0.0 and 1.0 and using that to choose an element from our list.  The underlying random number is being chosen using the system time to 'seed' the draw between 0.0 and 1.0 - that is why if you keep running the cell you obtain a different number.  

Let's set the seed and see what happens.

In [ ]:
#let's generate two bits of code to determine how random numbers are drawn

#1. repeat drawing but allowing the random seed to be drawn from the system time
print('test 1')
#explicitly set seed to empty, which defaults to system time
rand.seed()
for i in range(0,10):
    print(rand.randint(0,100))

#2. set random seed - why did I choose this number?
rand.seed(1209)

print('\ntest 2')
for i in range(0,10):
    print(rand.randint(0,100))

Run the above cell multiple times and you will see that the first set of numbers changes with every run.  However, the second set of numbers stays fixed.  In other words, if you know the seed (and random generator algorithm) that is being used to generate a set of random numbers, then the outcome is entirely deterministic.  This is the reason that Python's simple random package _should not_ be used for cryptographic purposes and that it calls its random numbers 'pseudo-random'. 

Let's now look at plotting some of the results of our random number generator so we can see the __probability distribution__.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#to visiualise a distribution we are going to have to draw from the distribution repeatedly.
# Let's initially set the number of draws to 40
Ndraws = 40

#initialise numpy array of zeros
x = np.zeros(Ndraws)
for i in range(0,Ndraws):
    x[i] = rand.randint(0,100)
    
#now plot the results as a histogram, setting the bin width to 1 in the left hand histogram so it is clear which numbers have been randomly selected
# leaving the bin width as automatic (=10 bins) to get a better sense of the shape of the distribution in the right hand plot
fg, ax = plt.subplots(nrows=1, ncols=2)

ax[0].hist(x, bins=100);
ax[1].hist(x)

#label axes
for a in ax:
    a.set_xlabel('Number drawn')

#set width of sub plots 
fg.set_figwidth(10)
plt.tight_layout()

You can see that the distribution looks quite lumpy.  If you re-run it, you will frequently find certain numbers that have been picked twice and many numbers that have been picked 0 times. 

To obtain a smoother distribution requires more sampling, so let's re-run with more draws from the distribution.  We will also switch the random number generator we are using to something a little more convenient to use.

In [ ]:
#let's use a numpy random number generator this time, so we can skip the for loop
Ndraws = 1000

x = np.random.randint(0,100,Ndraws)

fg, ax = plt.subplots(nrows=1, ncols=2)

ax[0].hist(x, bins=100);
ax[1].hist(x)

#let's also draw a horizontal line on, marking our prediction of what the underlying probability distribution looks like.
# why have I set the y-height as Ndraws/100 for the left hand plot?
ax[0].hlines(Ndraws/100,0,100, color='red')
# why have I set the y-height as Ndraws/100*10 for the right hand plot?
ax[1].hlines(Ndraws/100*10,0,100, color='red')

#label axes
for a in ax:
    a.set_xlabel('Number drawn')

#set width of sub plots 
fg.set_figwidth(10)
plt.tight_layout()

This should look smoother than the plots above.  In particular, the right hand plot that has wider bins and thereby averages the draws over several numbers, should be beginning to look more like a __[uniform distribution](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous))__.

Try changing the number of draws, how high do you have to set the number before you have a distribution that looks the same as our prediction?

## 6.2 Types of distributions
***

In the previous section we saw how to generate random integers, and how we can plot these to visualise the distribution we are drawing from.  

The uniform distribution we looked at is an important distribution, expressing equal preference (or weight) for numbers within its range, and zero weight for all numbers outside.  This is a useful distribution when you have some parameter that can vary between specific bounds, but not outside.  For example, the age of stars in our galaxy are constrained to be between 0 and the age of the universe.  Not knowing anything else, you might therefore consider their ages to be described by a uniform distribution.  

A second distribution that comes up time and time again is the __[normal distribution](https://en.wikipedia.org/wiki/Normal_distribution)__.  This is particular useful in data science because the uncertainty on data are often assumed (or known) to be normally distributed.  So, simulating the intrinsic noise on data can be achieved drawin from the normal distribution. 

Let's first look at the normal distribution, centred on 0 and with unit variance.

In [ ]:
#the numpy normal distribution takes arguments location, scale, size
# location = mean
# scale = 1 sigma
# size = number of points to be generated
mean = 0
sig = 1

#setup subplots
fg, ax = plt.subplots(nrows=1, ncols=2)
fg.set_figwidth(10)

#plot two sets of random draws, the right hand plot drawing 10 times more from the underlying distributino
# note that plotting a histogram returns a series of objects, look at the matplotlib.pyplot.hist page to 
# understand what these are.  For our purposes the one we want to access later is the bins (stored in b1 and b2), so we can calculate
# the bin width and use that to plot our underlying distribution at the correct y-axis height.
n, b1, p = ax[0].hist(np.random.normal(mean,sig,100))
n, b2, p = ax[1].hist(np.random.normal(mean,sig,1000))

#set common x-range to the plots at +/- 5 sigma
for a in ax:
    a.set_xlim(-5,5)

#plot on theoretical distribution, recalling the definition of a normal distribution
# np.linspace creates a linearly spaced series of numbers between limits in a numpy array.
x = np.linspace(-5,5,1000)
# create array of zeros to fill in
y = np.zeros(1000)
# enumerate just counts where we are through the array of x values
for i, xi in enumerate(x):
    y[i] = 1/(sig*(2*np.pi)**(0.5)) * np.exp(-0.5 * ((xi-mean)/sig)**2)

#now plot underlying normal distribution
ax[0].plot(x, y*100*(b1[1]-b1[0]), color='red')
ax[1].plot(x, y*1000*(b2[1]-b2[0]), color='red')


plt.tight_layout()

Note, that this is now a __continuous__ distribution.  Whilst previously we were drawing uniformly from integers between 0 and 100, our draws now lie randomly along the real number line concentrated around the mean of the distribution.  

There are many pre-computed __[distributions available](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html)__ for use through NumPy, which are useful for various statistical tests you may come across.  

## 6.3 Error propagation
***

Random number generation is enormously useful for error propagation.  Let's now look at an example of this using a real dataset and see how drawing from the normal distribution can be useful in quantifying our uncertainty in properties of the data.

In __[Exercise 5](Exercise5.ipynb)__ we look at fitting isochrons.  Let's revist this, but now using our knowledge of random sampling to estimate the uncertainty on the isochron age.  

First, we will replot the data, and compare it to a plot where the data are randomly purturbed according to their uncertainty.

In [ ]:
#-------------same as in Exercise 5---------------------------#
import scipy.stats as sts
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('./data/Sierra_nevada_granodiorite.txt',sep=',',header=0)

#calculate regression through data
res = sts.linregress(df.rbsr8786, df.srsr8786)

def lfunc(x, m, c):
    y = m*x + c
    return y

fg, ax = plt.subplots(nrows=1, ncols=2)

#make plot with error bars
ax[0].errorbar(df.rbsr8786, df.srsr8786, yerr=df.srsr_1sig, xerr=(df.rbsr_1sigpc*df.rbsr8786/100), 
            color='white', marker='o', markeredgecolor='black', linestyle='None',
           ecolor='black')

#-----------------------------------------------------#
#now, let's replot the data randomly purturbing their location according to their error

# first, we will plot the original data in the background so we can see how points have shifted
ax[1].scatter(df.rbsr8786, df.srsr8786, color='lightgrey', marker='o', linestyle='None')

#now let's generate our randomly shifted dataset
# first the x data, note we can pass the numpy random function arrays of mean values and standard deviations, so we can resample the data in one line
x = np.random.normal(df.rbsr8786, df.rbsr_1sigpc*df.rbsr8786/100)
# then the y data the same way
y = np.random.normal(df.srsr8786, df.srsr_1sig)

#Now plot the new data
ax[1].scatter(x,y, marker='o', color='red')

#tidying up appearance
for a in ax:
    a.set_ylim([0.706,0.708]);
    a.set_xlabel(r'${^{87}Rb/^{86}Sr}$')
    a.set_ylabel(r'${^{87}Sr/^{86}Sr}$');
    a.grid(ls=':');


fg.set_figwidth(10)
plt.tight_layout()


Notice that our randomly resampled data lie displced from the original data.  What's more, if you rerun the cell above you will see the data shift around each time, as a new draw from the distribution is made.

This might seem useful for visualisation purposes but otherwise a little useless.  However, the power of random sampling is in calculating how this uncertainty propagates through to a calculation we perform on the data.  For the isochron example what we care about is the age.  So let's now use random resampling to calculate the error on the ages that we obtain.

In [ ]:
fg, ax = plt.subplots(nrows=1, ncols=1)

#make plot with error bars
ax.errorbar(df.rbsr8786, df.srsr8786, yerr=df.srsr_1sig, xerr=(df.rbsr_1sigpc*df.rbsr8786/100), 
            color='white', marker='o', markeredgecolor='black', linestyle='None',
           ecolor='black', zorder=2)
xlim = ax.get_xlim()

#decay constant for Rb -> Sr decay; needed for calculating ages
lmbd = 1.393e-11

#Let's repeat the regression calculation 1000 times with a new and randomly purturbed dataset in each case.
# create list object to hold results of all the regression calculations
res = []
# create a numpy array to store the ages we calculate
ages = np.zeros(1000)
for i in range(0,1000):
    #generate new data
    x = np.random.normal(df.rbsr8786, df.rbsr_1sigpc*df.rbsr8786/100)
    y = np.random.normal(df.srsr8786, df.srsr_1sig)
    res.append(sts.linregress(x, y))
    
    ages[i] = np.log(res[-1].slope + 1)/lmbd

    #plot regression line using gradient and intercept just calculated
    # recall that [-1] accesses the last element in an list
    x = np.linspace(xlim[0], xlim[1], len(y))
    y = lfunc(x, res[-1].slope, res[-1].intercept)
    ax.plot(x, y, linestyle='-', color='lightgrey', zorder=1)
    
#we will overlay the original regression line, as calculated using the mothod of Exercise 5
res_orig = sts.linregress(df.rbsr8786, df.srsr8786)
x = np.linspace(xlim[0], xlim[1], 100)
y = lfunc(x, res_orig.slope, res_orig.intercept)
ax.plot(x,y, linestyle='-', color='black',zorder=1)

#now let's write the result onto the plot
text = 'age (random sampling): {:.1f} $\pm$ {:.0f} Myr'.format(np.mean(ages)/1e6, np.std(ages)/1e6)
ax.text(0.9,0.1, text, transform=ax.transAxes, horizontalalignment='right',verticalalignment='bottom')
# and put the original result on as well
text = 'age (raw data): {:.1f} Myr'.format((np.log(res_orig.slope + 1)/lmbd)/1e6)
ax.text(0.9,0.05, text, transform=ax.transAxes, horizontalalignment='right',verticalalignment='bottom')

ax.set_ylim([0.706,0.708]);
ax.set_xlabel(r'${^{87}Rb/^{86}Sr}$')
ax.set_ylabel(r'${^{87}Sr/^{86}Sr}$');
ax.grid(ls=':');

Notice how the average age from the 1000 random realisations is close (perhaps even the same) as the age obtained from the regression through the raw data.  What happens if you increase the number of random realisations?  Does it converge (and stablise) on the same average age as the regression of the raw data?

We have also plotted each of the regression lines generated, which gives a sense of the envelope of possible regression lines that could be fit through the data, given their uncertainty.

The analysis performed here is often called __Monte Carlo__ sampling, a technique __[born during the Manhattan project](https://en.wikipedia.org/wiki/Monte_Carlo_method#History)__.  Despite the name and the technique's history, it is no more complex than __repeat random sampling__.  Despite the simplicity, it is a very powerful tool in scientific computing.

<div class=obj>
    <b>Note:</b> We haven't performed a complete error analysis on this data.  Although far better than just reporting the age as 89.4 Myr with no uncertainty, we haven't accounted for the correlated error on x and y that arises from their common denominator.  In this case the correlated uncertainty is unlikely to make a significant difference, because the x error is so much smaller than the y error.  This isn't always the case, for example with U-Pb dating.
</div>
<p></p>

## 6.3 Random sampling of objects
***



The final example we will look at is random sampling of objects.  To illustrate this we will code up a classic gameshow gamble.  The setup is this:

> The host of the show sets up three identical boxes.  Two boxes are empty, and in the third is a beautiful piece of G5a, Shap granite, totally priceless.  The host knows where the specimen is and invites you to pick a box.  The host then picks _another_ box, opens it and shows you it is empty.  
>
> The host then gives you a choice, 'keep your first choice, or select the remaining box?'.
>
> What should you do?

Now, we could reason out the answer to this puzzle.  But we can code we don't have to reason! Let's get the computer to do it.

We are going to write code to randomly populate three boxes, randomly pick one box, and then vary between systematically switching the box we have picked vs. sticking with the original box we picked.  In both cases we will calculate our win percentage, and from this know what to do if ever faced with this high stakes gamble. 

In [ ]:
import random as rand

#setup boxes
box = ['sadness', 'sadness', 'shapp']
#setup static index array to box
boxid = [0, 1, 2]

#let's play this game a lot
Ngames = 1000
# we will need to count how many times we win
win_count = 0
for i in range(0,Ngames):
    #shuffle the box
    box = rand.sample(box,3)
    
    #pick our box
    pick = rand.sample(boxid,1)[0]

    #now the host needs to pick their box, and they will always pick and empty box
    for i, b in enumerate(box):
        if i != pick and b == 'sadness':
            show = i

    #now, we decide to stick or switch to the remaining box
    # let's first see what happens if we stick
    if box[pick] == 'shapp':
        win_count = win_count + 1

print('Sticking, we won', win_count/Ngames*100, ' percent of the time')

#now let's look at switching
# remembering to reset our win counter
win_count = 0
for i in range(0,Ngames):
    #reset the box, this method modifies the box each step, so it needs resetting
    box = ['sadness', 'sadness', 'shapp']
    #shuffle the box
    box = rand.sample(box,3)
    #reset box id's
    boxid = [0,1,2]
    
    #pick our box
    pick = rand.sample(boxid,1)[0]

    #now the host needs to pick their box, and they will always pick an empty box
    for i, b in enumerate(box):
        if i != pick and b == 'sadness':
            show = i

    #now, we decide to stick or switch to the remaining box
    # this time we switch
    # to code this we are going to delete the elements from the box we have picked and the host has 
    # showed us, just leaving us with the remaining box.
    for i in sorted([pick, show], reverse=True):
        del box[i]

    if box[0] == 'shapp':
        win_count = win_count + 1

print('Switching, we won', win_count/Ngames*100, ' percent of the time')

Can you make sense of this result?

In anycase, here is some digital G5a as a reward for getting this far.

![shapp](img/shap_granite_1.jpg)

# Independent coding
***



<div class=obj>
    <b>Aim:</b> To use a Monte Carlo methods to propagate error through a calculation.
</div>
<p></p>



A common problem where random numbers come in useful is when needing to propagate error through a calculation or series of calculations.  Rather than __[formal error propagation](https://en.wikipedia.org/wiki/Propagation_of_uncertainty)__ it is often easier to make the computer do all the work, and recalculate multiple times with the input randomly pertured. 

Let's look at this now for calculating the crystallisation temperature of magmatic olivines.  This topic has been an ongoing source of controversy, because of it being the first step in calculating mantle potential temperature and the debate over whether mantle plumes are hotter than ambient mantle (see __[Matthews et al. (2016)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016GC006497)__ for a recent perspective).  

A popular method for calculating magamatic temperatures uses the temperature dependent partitioning of Al between spinel and olivine to estimate the temperature crystals grow at.  The olivine-spinel Al partitioning has been experimentally investigated by __[Wan et al. 2008](https://doi.org/10.2138/am.2008.2758)__ and __[Coogan et al. 2014](http://dx.doi.org/10.1016/j.chemgeo.2014.01.004)__ leading to

\begin{align}
T(\mathrm{K}) = \frac{10^4}{0.575(0.162) + 0.884(0.043)Y_\mathrm{Cr} - 0.897(0.025)\ln(\mathrm{k_d})},
\end{align}

where the numbers in parentheses are the 1 sigma on the parameter estimate.  $Y_\mathrm{Cr}$ is the Cr number of the spinel, and is given by 

\begin{align}
Y_\mathrm{Cr}= \frac{X_\mathrm{Cr}}{(X_\mathrm{Cr}+X_\mathrm{Al})},
\end{align}

where $X_i$ is the _atomic_ abundance of element $i$.  Atomic abundances can be calculated from weight fractions by dividing each oxide component present in the mineral's composition by their mean molecular mass and renomalising.  It is important to do this on a single cation basis, i.e., $\mathsf{AlO_{1.5}}$ rather than $\mathsf{Al_2O_3}$ as the composition will be reported in the raw data, as $X_i$ is counting atoms and there are two Al in every $\mathsf{Al_2O_3}$.  Expressing this mathematically,

\begin{align}
X_i = \frac{w_i/m_i}{\sum{w_j/m_j}},
\end{align}

where $w_i$ is the weight fraction (or percent) of the oxide, which is how raw compositional data will typically be reported, and $m_i$ is the mean molecular mass of the molecule given in the table below.

Finally, $k_d$ is the exchange coefficient of Al between the spinel and olivine and is given by

\begin{align}
\mathrm{k_d} = \frac{\mathsf{Al_2O_3^{olivine}}}{\mathsf{Al_2O_3^{spinel}}},
\end{align}

where the composition here is used as weight percent (i.e., how it will be given in the raw compositional data).

|molecule| mean molecular mass |
|-----------|------------|
|SiO$_2$    | 60.0 |
|MgO        | 40.3 |
|AlO$_{1.5}$| 51.0 |
|TiO$_2$    | 79.9 |
|CaO        | 56.0 |
|FeO        | 71.8 |
|CrO$_{1.5}$| 76.0 |
|MnO        | 70.9 |
|NiO        | 74.7 |

Now, we will read in a data file with compositional information from two olivine-spinel pairs, one from Hawaii and one from Iceland, calculate their crystallisation temperatures propagating error on the experimental parameters and compositional data, and decide whether they record different magmatic temperatures between the two settings.  The awkward part of all of this is managing the two data files and making the right parts line up during the calculations -- and it really is awkward, but is a very real challenge in the datasciences. The steps to take are:

- Read in the compositional data (file: `crystal_ts.xlsx` using pandas `read_excel`; data are weight percent).
- Read in the data file containing the molecular weights (on a single cation basis; file: `molecular_weights.txt` using `read_csv`)
- Now, separate the calculation into separate parts.  The simplest thing to do is string all the steps together directly.
    - First you will need to take the compositions read in (for olivine and spinel) and randomly perturb them according the error reported in the data files.
    - Then calculate the atomic fractions of Cr and Al in the spinel so that you can calculate $Y_\mathrm{Cr}$.
    - Then calculate the $\mathrm{k_d}$ from the olivine and spinel compositions (using the raw weight percent data of the data file).
    - Now perturb the parameters in the themometry equation by their error.
    - Finally, calculate the temperature and store it in an array.
- Run through that sequence of calculations a large number of times, and end by calculating the mean and standard deviation.

### Going further 
The hard part of this exercise is finding an elegant solution to the data manipulation and calculations that need to be performed.  An inelegant solution will work fine and is probably quicker to code.  But if you want to go further, look into the __[Pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide)__ to find out how built in features of the data frames could be used to your advantage.